# Discretization

In this notebook I describe how to translate the mathematical basics into a form, that makes numeric programming possible. This requires discretization in time. For further details I recommend [Trajectory Planning for BERTHA - a Local, Continuous Method](https://pdfs.semanticscholar.org/bdca/7fe83f8444bb4e75402a417053519758d36b.pdf).

In [23]:
import numpy as np
from scipy.optimize import minimize
from matplotlib import pyplot as plt

## Discretization of the kinematic model

First of all the function $s(t)$ has to be discretized to a sequence in which $s_{i}$ describe the value of $s$ during discrete moments $i$. These moments are equidistant in time. The distance between adjacent points in time is $\Delta t = \frac{T}{N-1}$, where $T$ is the optimization timespan and $N$ is the desired number of points. These numbers may be chosen freely. I use $ T = 10 s$ and $ N = 30 $.

Due to the discretization the velocity, accelleration and jerk cannot be computed as derivates of $s$. They have to be adjusted to finite differences:

$$ v_{i} \approx \frac{s_{i+1} - s_{i}}{\Delta t} \quad i = 0,...,N-1 $$
-
$$ a_{i} \approx \frac{v_{i+1} - v_{i}}{\Delta t} \quad i = 0,...,N-2 $$
-
$$ j_{i} \approx \frac{a_{i+1} - a_{i}}{\Delta t} \quad i = 0,...,N-3 $$

In [6]:
T = 10
N = 30

del_t = T/(N-1)

# Velocity
def v(x):
    
    v = np.zeros(N - 1)
    v = (x[1:] - x[:-1])/del_t
    return v

# Accelleration
def a(x):
    v1 = v(x)
    
    a = np.zeros(N - 2)
    a = (v1[1:] - v1[:-1])/del_t
    return a

# Jerk
def j(x):
    a1 = a(x)
    
    j = np.zeros(N - 3)
    j = (a1[1:] - a1[:-1])/del_t
    return j

## Discretization of the cost functional

Due to the discretization of $s$ the argument of the cost functional

$$ J(s(t)) = \frac{1}{2} \int_{t}^{t+T} w_{v}(v(t) - v_{des})^{2} + w_{a}a(t)^{2} + w_{j}j(t)^{2} dt $$


is not a function in time, but a sequence. This means that the cost functional is simplified to the cost function

$$ J(s_{0},...,s_{N-1}) = \frac{1}{2} \left ( w_{v}\sum_{i=0}^{N-1} (v_{i} - v_{des})^{2} + w_{a}\sum_{i=0}^{N-2} a_{i}^{2} + w_{j}\sum_{i=0}^{N-3} j_{i}^{2} \right ) \Delta t. $$

The integral can be simplified to a sum. 

In [3]:
def J(x):
    return 0.5*(sum((v(x) - v_des)**2) + sum(a(x)**2) + sum(j(x)**2))*del_t

## Physical Constraints

The phyiscal constraints 

$$ v(t) \le v_{max}, $$
-
$$ a(t) \le a_{max}. $$

are added to the optimizer as 

In [4]:
v_max = 30
a_max = 15

def cons_vel(x):
    return v_max - v(x)

def cons_acc(x):
    return a_max - a(x)

def pos_ves(x):
    return 

## Bound Constraints

Since the cost function $J(s_{0},...,s_{N-1})$ is only proportional to derivations of $s$, but not to $s$ itself, the optimization will lead a trajectory that is located arbitrarilly. To prevent this, bound constraints are used to fixate the trajectories location. Furthermore, the trajectories first points are predetermined because of the self driving cars location, velocity and accelleration. 

In general, bound contraints are used to limit each optimization variable. In this case $N$ bound constraints are needed, because the optimization problem contains $N$ optimization variables $s_{i}$ with $i = 0,..., N-1$.

In [16]:
vel_init = 0
acc_init = 0

bounds = [(0, np.inf) for i in range(N)]
bounds[0] = (0, 0)
bounds[1] = (vel_init * del_t, vel_init * del_t)
bounds[2] = (acc_init * del_t ** 2 + 2 * vel_init * del_t, acc_init * del_t ** 2 + 2 * vel_init * del_t)

The provided information already suffices to compute a trajectory without obstacles. An initialization is required for the optimizer. It can be chosen randomly. The desired velocity $v_{des}$ is provided as well.

In [12]:
x_0 = np.arange(30)
v_des = 20

In [17]:
no_obstacles_res = minimize(J, x_0, method='SLSQP', bounds = bounds, options={'disp': True})

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 420.523805194
            Iterations: 36
            Function evaluations: 1233
            Gradient evaluations: 36


In [ ]:
t_axis = np.linspace(0, 10, N)

plt.plot(t_axis, no_obstacles_res.x)
plt.grid()
plt.show()

To see the result of the optimization in a plot, run all the cells of the notebook upto this point!

## Obstacles

![](img/small.png)

Obstacles, like the red car in the image above are also represented in the bound constraints. To be precise, the linear constraints 

$$ s(t) \ge s_{b}, \quad \in [t_{a}, t_{b}], $$
-
$$ s(t) \le s_{a}, \quad \in [t_{a}, t_{b}], $$

are transformed to 

$$ s_{i} \ge s_{b}, \quad i = t_{a},..., t_{b}, $$
-
$$ s_{i} \le s_{a}, \quad i = t_{a},..., t_{b}. $$

Because both equations describe different convex spaces, two optimizations have to be performed (one with each equation worked into the bound constraints). The implementation details will follow...